## Import required lib

In [1]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 49.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 22.7 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 11.9 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.6 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ToTensor, Normalize
import nibabel as nib
from scipy.ndimage import zoom
import torch.nn as nn
from torchvision.models.video import r3d_18
import pandas as pd
import SimpleITK as sitk
import pickle

## Load the excel file with paths

In [5]:
final_df = pd.read_csv('/kaggle/input/output-excel/final_output.csv')
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2113 entries, 0 to 2112
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Subject        2113 non-null   object
 1   Age            2113 non-null   int64 
 2   Modality       2113 non-null   object
 3   Group          2113 non-null   object
 4   Image Data ID  2113 non-null   int64 
 5   path           2113 non-null   object
 6   filename       2113 non-null   object
dtypes: int64(2), object(5)
memory usage: 115.7+ KB


In [6]:
final_df

,Subject,Age,Modality,Group,Image Data ID,path,filename
0,128_S_0272,71,MRI,CN,68938,/kaggle/input/adnidata/ADNI/128_S_0272/MPR-R__...,ADNI_128_S_0272_MR_MPR-R__GradWarp__B1_Correct...
1,128_S_0272,73,MRI,CN,105494,/kaggle/input/adnidata/ADNI/128_S_0272/MPR-R__...,ADNI_128_S_0272_MR_MPR-R__GradWarp__B1_Correct...
2,128_S_0272,72,MRI,CN,69226,/kaggle/input/adnidata/ADNI/128_S_0272/MPR-R__...,ADNI_128_S_0272_MR_MPR-R__GradWarp__B1_Correct...
3,128_S_0272,71,PET,CN,57334,"/kaggle/input/adnidata/ADNI/128_S_0272/Coreg,_...","ADNI_128_S_0272_PT_Coreg,_Avg,_Standardized_Im..."
4,128_S_0272,74,PET,CN,149234,"/kaggle/input/adnidata/ADNI/128_S_0272/Coreg,_...","ADNI_128_S_0272_PT_Coreg,_Avg,_Standardized_Im..."
...,...,...,...,...,...,...,...
2108,029_S_1073,67,PET,MCI,1596802,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."
2109,029_S_1073,67,PET,MCI,1596748,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."
2110,029_S_1073,66,PET,MCI,1596851,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."
2111,029_S_1073,68,PET,MCI,1596813,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."


In [11]:
mri_df = final_df[final_df['Modality'] == 'MRI']
mri_paths = (mri_df['path'] +'/'+mri_df['filename']).to_numpy()
mri_labels = mri_df['Group'].to_numpy()

pet_df = final_df[final_df['Modality'] == 'PET']
pet_paths = (pet_df['path'] +'/'+pet_df['filename']).to_numpy()
pet_labels = pet_df['Group'].to_numpy()

## Dataloader to create the dataset and split into test and train images

In [12]:
'''
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        #image = nib.load(image_path).get_fdata()
        #print(image.size())
        image = sitk.ReadImage(image_path)
        image = sitk.GetArrayFromImage(image)
        if self.transform is not None:
            image = self.transform(image)

        return image, label

# Define your image paths and labels as numpy arrays
image_paths = paths  # Replace with your own image paths
labels = labels  # Replace with your own labels
'''
#MRI
# Perform 70-30 train-test split
mri_train_image_paths, mri_test_image_paths, mri_train_labels, mri_test_labels = train_test_split(
    mri_paths, mri_labels, test_size=0.3, random_state=42)


#PET
pet_train_image_paths, pet_test_image_paths, pet_train_labels, pet_test_labels = train_test_split(
    pet_paths, pet_labels, test_size=0.3, random_state=42)

In [13]:
# print(mri_paths.shape)
# print(mri_train_image_paths.shape)
# print(mri_test_image_paths.shape)


# print(pet_paths.shape)
# print(pet_train_image_paths.shape)
# print(pet_test_image_paths.shape)


In [14]:
def pathstoarray(train_image_paths):
    train_images = []
    for image_path in train_image_paths:
        # Read image using OpenCV
        image = sitk.ReadImage(image_path)
        image = sitk.GetArrayFromImage(image)
        #print(image.shape)
        # Convert BGR image to RGB
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Append image to list
        image = np.resize(image, (60, 128, 128))
        train_images.append(image)
    return train_images

In [15]:
# mri_train_images = np.array(pathstoarray(mri_train_image_paths))
# mri_test_images = np.array(pathstoarray(mri_test_image_paths))

In [16]:
# pet_train_images = np.array(pathstoarray(pet_train_image_paths))
# pet_test_images = np.array(pathstoarray(pet_test_image_paths))

In [17]:
# print(mri_train_images.shape)
# print(mri_test_images.shape)

# print(pet_train_images.shape)
# print(pet_test_images.shape)


In [18]:
# mri_train_pkl = 'mri_train_array.pkl'
# with open(mri_train_pkl, 'wb') as f:
#     pickle.dump(mri_train_images, f)

# mri_test_pkl = 'mri_test_array.pkl'
# with open(mri_test_pkl, 'wb') as f:
#     pickle.dump(mri_test_images, f)
    
# pet_train_pkl = 'pet_train_array.pkl'
# with open(pet_train_pkl, 'wb') as f:
#     pickle.dump(pet_train_images, f)

# pet_test_pkl = 'pet_test_array.pkl'
# with open(pet_test_pkl, 'wb') as f:
#     pickle.dump(pet_test_images, f)

In [19]:
with open('/kaggle/input/pickle-files/mri_train_array.pkl', 'rb') as f:
    mri_train_images = pickle.load(f)

with open('/kaggle/input/pickle-files/mri_test_array.pkl', 'rb') as f:
    mri_test_images = pickle.load(f)
    
with open('/kaggle/input/pickle-files/pet_train_array.pkl', 'rb') as f:
    pet_train_images = pickle.load(f)

with open('/kaggle/input/pickle-files/pet_test_array.pkl', 'rb') as f:
    pet_test_images = pickle.load(f)

print(mri_train_images.shape)
print(mri_test_images.shape)

print(pet_train_images.shape)
print(pet_test_images.shape)


(510, 60, 128, 128)
(219, 60, 128, 128)
(968, 60, 128, 128)
(416, 60, 128, 128)


In [20]:
classes = np.array(['MCI', 'CN', 'AD'])
# Create a dictionary to map class names to their corresponding index values
class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}
# Use a list comprehension to convert the class names to index values
mri_labels_idx = np.array([class_to_idx[label] for label in mri_train_labels])
# Use the index values to create the one-hot encoded labels
mri_lb_oh = np.eye(len(classes))[mri_labels_idx]
mri_train_data_model = mri_train_images[:, np.newaxis, :,:,:]


pet_labels_idx = np.array([class_to_idx[label] for label in pet_train_labels])
pet_lb_oh = np.eye(len(classes))[pet_labels_idx]
pet_train_data_model = pet_train_images[:, np.newaxis, :,:,:]

In [21]:
print(mri_train_data_model.shape)
print(mri_lb_oh.shape)

print(pet_train_data_model.shape)
print(pet_lb_oh.shape)

(510, 1, 60, 128, 128)
(510, 3)
(968, 1, 60, 128, 128)
(968, 3)


## RESNET model implementation

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, ZeroPadding3D,\
     Flatten, BatchNormalization, AveragePooling3D, Dense, Activation, Add  , Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import optimizers

D0419 05:37:05.696332975      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0419 05:37:05.696368868      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0419 05:37:05.696372631      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0419 05:37:05.696375567      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0419 05:37:05.696378155      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0419 05:37:05.696381230      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0419 05:37:05.696383890      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0419 05:37:05.

In [23]:
def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv3D(f1, kernel_size=(1,1, 1), strides=(1, 1,1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv3D(f1, kernel_size=(3,3, 3), strides=(1,1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv3D(f2, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv3D(f1, kernel_size=(1,1, 1), strides=(s,s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv3D(f1, kernel_size=(3,3, 3), strides=(1,1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv3D(f2, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv3D(f2, kernel_size=(1, 1,1), strides=(s,s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [24]:
def resnet50():

  input_im = Input(shape=(1,60, 128, 128,)) # cifar 10 images size,
  x = ZeroPadding3D(padding=(1,1, 1))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv3D(64, kernel_size=(3, 3,3), strides=(1,1, 1))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling3D((1,1, 1), strides=(1, 1,1))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))

  # 3rd stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 4th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 5th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # ends with average pooling and dense connection

  x = AveragePooling3D((2,2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(3, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [25]:
base_model_PET = resnet50()

fc_layer1 = tf.keras.layers.Dense(512, activation='relu')(base_model_PET.layers[-2].output)
fc_layer1 = tf.keras.layers.Dropout(0.8)(fc_layer1)
output_layer = tf.keras.layers.Dense(3, activation='sigmoid')(fc_layer1)
PETmodel = Model(inputs=base_model_PET.input, outputs=output_layer)

In [34]:
opt = optimizers.Adam(learning_rate=0.001)
PETmodel.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
PETmodel.fit(pet_train_data_model,pet_lb_oh,batch_size=16,epochs=20,verbose=1)

Epoch 1/20
61/61 [==============================] - 151s 2s/step - loss: 3.1756 - accuracy: 0.5589
Epoch 2/20
61/61 [==============================] - 129s 2s/step - loss: 2.3281 - accuracy: 0.5682
Epoch 3/20
61/61 [==============================] - 129s 2s/step - loss: 1.9728 - accuracy: 0.5713
Epoch 4/20
61/61 [==============================] - 130s 2s/step - loss: 1.7256 - accuracy: 0.5733
Epoch 5/20
61/61 [==============================] - 129s 2s/step - loss: 1.6162 - accuracy: 0.5661
Epoch 6/20
61/61 [==============================] - 128s 2s/step - loss: 1.5848 - accuracy: 0.5775
Epoch 7/20
61/61 [==============================] - 129s 2s/step - loss: 1.5026 - accuracy: 0.5671
Epoch 8/20
61/61 [==============================] - 129s 2s/step - loss: 1.4814 - accuracy: 0.5661
Epoch 9/20
61/61 [==============================] - 129s 2s/step - loss: 1.5636 - accuracy: 0.5692
Epoch 10/20
61/61 [==============================] - 128s 2s/step - loss: 1.5026 - accuracy: 0.5723
Epoch 11/

In [35]:
pet_test_labels_idx = np.array([class_to_idx[label] for label in pet_test_labels])
pet_test_data_model = pet_test_images[:, np.newaxis, :,:,:]
print(pet_test_data_model.shape)

(416, 1, 60, 128, 128)


In [36]:

y_pred_probs = PETmodel.predict(pet_test_data_model)  # Predicted probabilities for each class
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Compute the confusion matrix
confusion_mat = confusion_matrix(pet_test_labels_idx, y_pred)

# Compute evaluation metrics
accuracy = accuracy_score(pet_test_labels_idx, y_pred)
precision = precision_score(pet_test_labels_idx, y_pred, average='weighted')
recall = recall_score(pet_test_labels_idx, y_pred, average='weighted')
f1score = f1_score(pet_test_labels_idx, y_pred, average='weighted')

print("Confusion Matrix:\n", confusion_mat)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1score))

13/13 [==============================] - 7s 408ms/step
Confusion Matrix:
 [[240  11   0]
 [105  10   0]
 [ 50   0   0]]
Accuracy: 0.6010
Precision: 0.4982
Recall: 0.6010
F1-score: 0.4890


/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
print(y_pred_probs)

[[0.63844955 0.50145686 0.29417235]
 [0.1141073  0.9787804  0.5527221 ]
 [0.638449   0.50145626 0.29417205]
 ...
 [0.638501   0.501458   0.2940642 ]
 [0.6384491  0.5014564  0.29417208]
 [0.63844913 0.5014565  0.29417214]]


In [ ]:
base_model_MRI =resnet50()

fc_layer = tf.keras.layers.Dense(512, activation='relu')(base_model_MRI.layers[-2].output)
fc_layer = tf.keras.layers.Dropout(0.8)(fc_layer)
fc_layer = Model(inputs=base_model_MRI.input, outputs=fc_layer)


base_model_PET = resnet50()

fc_layer1 = tf.keras.layers.Dense(512, activation='relu')(base_model_PET.layers[-2].output)
fc_layer1 = tf.keras.layers.Dropout(0.8)(fc_layer1)
fc_layer1 = Model(inputs=base_model_PET.input, outputs=fc_layer1)

combined = Concatenate()([fc_layer.output, fc_layer1.output])
z = Dense(2, activation="relu")(combined)
output_layer = tf.keras.layers.Dense(3, activation='sigmoid')(z)
model = tf.keras.models.Model(inputs=[base_model_MRI.input,base_model_PET.input], outputs=output_layer)

In [ ]:
#model.summary()

In [ ]:
data = train_data_model
labels = one_hot_train_labels

In [ ]:
model.compile('adam','mse')

In [ ]:
model.fit([data,data],labels,batch_size=16,epochs=2,verbose=1)

In [ ]:
test_images = []
for image_path in test_image_paths:
    # Read image using OpenCV
    image = sitk.ReadImage(image_path)
    image = sitk.GetArrayFromImage(image)
    #print(image.shape)
    # Convert BGR image to RGB
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Append image to list
    image = np.resize(image, (60, 128, 128))
    test_images.append(image)

In [ ]:
test_images = np.array(test_images)
print(test_images.shape)


# Use a list comprehension to convert the class names to index values
test_labels_idx = np.array([class_to_idx[label] for label in test_labels])

# Use the index values to create the one-hot encoded labels
one_hot_test_labels = np.eye(len(classes))[test_labels_idx]
test_data_model = test_images[:, np.newaxis, :,:,:]
print(test_data_model.shape)
print(test_labels_idx)


In [ ]:
y_pred_probs = model.predict([test_data_model,test_data_model])  # Predicted probabilities for each class
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Compute the confusion matrix
confusion_mat = confusion_matrix(test_labels_idx, y_pred)

# Compute evaluation metrics
accuracy = accuracy_score(test_labels_idx, y_pred)
precision = precision_score(test_labels_idx, y_pred, average='weighted')
recall = recall_score(test_labels_idx, y_pred, average='weighted')
f1score = f1_score(test_labels_idx, y_pred, average='weighted')

print("Confusion Matrix:\n", confusion_mat)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1score))

In [ ]:
print(test_labels_idx)

In [ ]:
#print(test_data_model)